In [4]:
print("Hello World")

Hello World


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
COHERE_API_KEY=os.getenv("COHERE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
MODEL="mixtral-8x7b-32768"

In [6]:
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings

model=ChatGroq(
    api_key=GROQ_API_KEY,
    model=MODEL,
)
embeddings=CohereEmbeddings()
model.invoke("Make me laugh")

AIMessage(content="Sure thing! Here's a joke for you:\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!\n\nI hope that made you laugh! Do you have any favorite jokes you'd like to share? I'm always up for a good laugh.", response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 13, 'total_tokens': 80, 'completion_time': 0.102627362, 'prompt_time': 0.003803685, 'queue_time': None, 'total_time': 0.106431047}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-b535f95c-0a95-426b-91a9-bc38a666b13d-0')

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Tell me a joke"))

Sure! Here's one:

Why couldn't the bicycle stand up by itself?

Because it was two-tired!

Do you have any jokes you'd like to share? I'd love to hear them!


In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("data/guideline-170-en.pdf")
pages=loader.load_and_split()
pages

[Document(page_content='Page 1 / 394Exported on: 04/04/2024\nClinical guidelines - Diagnosis and treatment\nmanual\nFor curative programmes in hospitals and dispensaries\nGuidance for prescribing\n\xa0\n\xa0\n© Médecins Sans Frontières\nAll rights reserved for all countries. No reproduction, translation and adaptation may be done without the prior\npermission of the Copyright owner.', metadata={'source': 'data/guideline-170-en.pdf', 'page': 0}),
 Document(page_content='Page 2 / 394ISBNMédecins Sans Frontières. Clinical guidelines - Diagnosis and treatment manual.\nMarch 2024\n978-2-37585-253-8', metadata={'source': 'data/guideline-170-en.pdf', 'page': 1}),
 Document(page_content='Page 3 / 394Table of contents\nAuthors/Contributors\nPreface\nAbbreviations and acronyms\nChapter 1: A few symptoms and syndromes\nChapter 2: Respiratory diseases\nChapter 3: Gastrointestinal disorders\nChapter 4: Skin diseases\nChapter 5: Eye diseases\nChapter 6: Parasitic diseases\nChapter 7: Bacterial disea

In [9]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [10]:
chain.input_schema.schema()

{'title': 'ChatGroqInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'t

In [11]:
chain = prompt | model | parser

chain.invoke(
    {
        "context": "I am naming you as SnoopDogg",
        "question": "What's your name?"
    }
)

'Based on the context, my name is SnoopDogg.'

In [12]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [13]:
retriever = vectorstore.as_retriever()
retriever.invoke("hypertension")

[Document(page_content='Page 379 / 394Essential hypertension in adults\nHypertension (or high blood pressure - HBP) is deﬁned as elevated blood pressure (BP) at rest that persists over time\ni.e. measured 3 times during 3 separate consultations over a period of three months.\nEssential hypertension is deﬁned as HBP of undetermined cause (the large majority of cases).\n\xa0\nThe global overall prevalence of HBP in adults aged 25 and over is around 40%.\n\xa0\nSerious complications of HBP can be acute (hypertensive encephalopathy, left-sided heart failure, acute renal failure)\nor delayed i.e. occur after a long period during which HBP has not been controlled (stroke, ischaemic heart disease,\nperipheral arterial disease, chronic renal impairment).\n\xa0\nFor pregnancy-induced hypertension, see\xa0Essential obstetric and newborn care, MSF.\nClinical features\n\xa0\n\xa0\nSBP\xa0≥ 180 and/or DBP ≥\xa0110 and some symptoms (moderate headaches, epistaxis, dizziness, tinnitus,\xa0eye\nﬂoater

In [14]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [16]:
questions = [
    "I have cold extremties, pale skin, and I feel cold, what could be the problem?",
    "Describe acute heart failure and its clinical features",
    "What is schistosomiases?",
    "What is the treatment of cellulitis?",
    "I have cough, wheezing and low-grade fever, what is my diagnosis?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: I have cold extremties, pale skin, and I feel cold, what could be the problem?
Answer: Based on the symptoms you've provided - cold extremities, pale skin, and feeling cold - it is possible that you could be experiencing symptoms of anemia. Anemia is a condition characterized by a low level of hemoglobin in the blood, which can lead to symptoms such as pallor (pale skin), fatigue, and feeling cold. However, it is important to note that these symptoms can also be caused by other medical conditions. Therefore, it is recommended that you consult with a healthcare professional for an accurate diagnosis and appropriate treatment.

Question: Describe acute heart failure and its clinical features
Answer: Acute heart failure, also known as acute pulmonary edema, is a sudden and life-threatening form of heart failure. It is characterized by the inability of the heart to maintain adequate cardiac output, resulting in the accumulation of fluid in the lungs.

The clinical features of acu

In [42]:
chain.batch([{"question": q} for q in questions])

["Based on the context provided, Lucy is a character who used to be conflicted and has now found her purpose. She is using her superpowers to save people, sometimes going beyond the limit and causing unintended consequences. Her current objective is to save a boy named Humblesmith, who is in possession of a plant with special powers and is being hunted by his clone. She is aware that the boy's life and the plant's fate are intertwined and will not be the same again.",
 "The encounter between Humblesmith and the haberdasher occurs in a document (page 1) when Humblesmith is on a hunting trip with a plant he found. He comes across a haberdasher and shows him the plant, but the haberdasher is not interested and tells Humblesmith to buy what he needs and leave quickly because a heavy rain is coming. The haberdasher appears to be uncomfortable and afraid to approach Humblesmith, repeatedly looking into the distance as if he can see Humblesmith's thoughts. Despite this, he urges Humblesmith t

In [46]:
for s in chain.stream({"question": "Who is Lucy and what is her main objective?"}):
    print(s, end="", flush=True)

Based on the context provided, Lucy is a character who used to be conflicted and has spent nine years saving her own life. Now, she is saving everyone she can, even "crossing the line." Her spirit is currently directing her to save a boy with a certain sentient plant, especially from his clone sent to kill him before he turns fourteen. One of Lucy's notable abilities is harnessing and controlling invisible magnetic energy beams. She previously saved a man named Jon Watts from committing suicide by pulling him off a ledge using her energy beams. Lucy's main objective at the moment is to save Humblesmith, the boy with the sentient plant.

In [47]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents